# **Vietnamese Medical Q&A Chatbot with BARTpho**

In [43]:
# from google.colab import drive
# drive.mount('/content/drive')
# #/content/drive/MyDrive/

In [44]:
pip install evaluate dataset


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [45]:
# pip numpy -version

## 1. Import Libraries

In [46]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from tqdm import tqdm
import logging
import pandas as pd

import collections

# Thư viện từ PyTorch
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import PreTrainedTokenizerFast, AutoModelForQuestionAnswering, AdamW, get_scheduler, default_data_collator

# Thư viện từ tqdm
from tqdm.auto import tqdm

# Thư viện đánh giá
import evaluate

# Các thư viện phụ trợ khác (nếu cần)
from typing import List, Dict


In [47]:
# Setup logging
logging.basicConfig(level=logging.INFO)

In [48]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [49]:
# device = torch.device("cpu"), mb25,

## 2. Load Pre-trained Model and Tokenizer

In [50]:
# Load BARTpho tokenizer and model
model_name = 'vinai/bartpho-word-base'

try:
    tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name)
    print(tokenizer.is_fast)
    model = AutoModelForQuestionAnswering.from_pretrained(model_name).to(device)
    print("Model and tokenizer loaded successfully.")
except Exception as e:
    print(f"An error occurred while loading the model: {e}")
    raise e

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'PhobertTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


True


Some weights of MBartForQuestionAnswering were not initialized from the model checkpoint at vinai/bartpho-word-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model and tokenizer loaded successfully.


In [51]:
# Nếu pad_token không có, thêm pad_token
if tokenizer.pad_token is None:
    # Thêm pad_token mới vào tokenizer
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})


## 3. Load DataFrame with Q&A Pairs

In [52]:
# Đọc dữ liệu từ file CSV hoặc JSON
df = pd.read_csv('/kaggle/input/ute-de-medical/processed_medical_full.csv')  # Thay đổi đường dẫn đến tập dữ liệu của bạn

# df = df.head(50000)

# Kiểm tra dữ liệu
df.head()




,Title,Detailed Content,Answer,Keywords,Question,Answer_Start,Answer_End,Reference Link
0,rào_cản tự_nhiên chống lại nhiễm_trùng về da,da thường chặn các vi_sinh_vật xâm_nhập trừ kh...,da thường chặn các vi_sinh_vật xâm_nhập trừ kh...,"da, vi_sinh_vật, động_vật, ống, iv, chặn, xâm_...",Liệu da có vai trò gì trong rào_cản tự_nhiên c...,0,139,https://www.msdmanuals.com/vi-vn/professional/...
1,rào_cản tự_nhiên chống lại nhiễm_trùng về da,da thường chặn các vi_sinh_vật xâm_nhập trừ kh...,da thường chặn các vi_sinh_vật xâm_nhập trừ kh...,"da, vi_sinh_vật, động_vật, ống, iv, chặn, xâm_...",Tại sao vi_sinh_vật lại xâm_nhập trong bối cản...,0,139,https://www.msdmanuals.com/vi-vn/professional/...
2,rào_cản tự_nhiên chống lại nhiễm_trùng về da,da thường chặn các vi_sinh_vật xâm_nhập trừ kh...,da thường chặn các vi_sinh_vật xâm_nhập trừ kh...,"da, vi_sinh_vật, động_vật, ống, iv, chặn, xâm_...",trừ động_vật có ảnh hưởng như thế nào đến rào_...,0,139,https://www.msdmanuals.com/vi-vn/professional/...
3,rào_cản tự_nhiên chống lại nhiễm_trùng về da,da thường chặn các vi_sinh_vật xâm_nhập trừ kh...,các ngoại_lệ bao_gồm virus papilloma_người có_...,"ngoại_lệ, virus, papilloma_người, da, mụn, cóc...",Sự kết hợp giữa ngoại_lệ và ngoại_lệ có thể ba...,140,479,https://www.msdmanuals.com/vi-vn/professional/...
4,rào_cản tự_nhiên chống lại nhiễm_trùng về da,da thường chặn các vi_sinh_vật xâm_nhập trừ kh...,các ngoại_lệ bao_gồm virus papilloma_người có_...,"ngoại_lệ, virus, papilloma_người, da, mụn, cóc...",Tại sao virus lại cần xâm_nhập trong rào_cản t...,140,479,https://www.msdmanuals.com/vi-vn/professional/...


In [53]:
import os

In [54]:
# # Đường dẫn đến thư mục mà bạn muốn tạo
# data_dir = '/kaggle/working/data'

# # Tạo thư mục nếu chưa tồn tại
# os.makedirs(data_dir, exist_ok=True)

# # Lưu DataFrame vào thư mục đã tạo
# df.to_csv(os.path.join(data_dir, 'data.csv'), index=False)

In [55]:
# df = df.head(5000)

df.shape

(164289, 8)

## 4. Data Preprocessing

In [56]:
# Data Preparation
titles = df['Title'].tolist()
detailed_contents = df['Detailed Content'].tolist()

# Đảm bảo tất cả các phần tử trong danh sách là chuỗi
titles = [str(title) for title in titles]
detailed_contents = [str(content) for content in detailed_contents]

In [57]:
import re

# Hàm tìm từ ghép có dấu '_'
def find_compound_words(texts):
    compound_words = set()
    for text in texts:
        # Tìm các từ chứa dấu '_'
        matches = re.findall(r'\b\w+_\w+\b', text)
        compound_words.update(matches)
    return list(compound_words)

# Tìm từ ghép trong titles và detailed_contents
compound_words_titles = find_compound_words(titles)
compound_words_contents = find_compound_words(detailed_contents)

# Kết hợp từ ghép từ cả titles và detailed_contents và đảm bảo tính duy nhất
compound_words = list(set(compound_words_titles + compound_words_contents))

# Kiểm tra số lượng từ ghép đã tìm thấy và các từ là duy nhất
print(f"Number of unique compound words: {len(compound_words)}")
print(compound_words[:10])  # In ra 10 từ ghép đầu tiên để kiểm tra

Number of unique compound words: 10681
['gương_lồi', 'hối_lỗi', 'ỉa_đùn', 'tiêu_cực', 'phối_hợp_hóa_trị', 'thắt_cổ', 'trầm_trọng', 'tuyến_tụy', 'dis_primers', 'đầu_lưỡi']


In [58]:
num_added_tokens = tokenizer.add_tokens(compound_words)

model.resize_token_embeddings(len(tokenizer))

print(f"Added {num_added_tokens} tokens. New vocab size: {len(tokenizer)}")

Added 10681 tokens. New vocab size: 76789


In [59]:
test_sentence = "da thường chặn các vi_sinh_vật xâm_nhập trừ khi nó bị tổn_thương ví_dụ do động_vật chân_đốt chấn_thương ống thông iv hoặc phẫu_thuật rạch . các ngoại_lệ bao_gồm virus papilloma_người có_thể xâm_nhập vào da bình_thường gây ra mụn cóc một_số ký_sinh_trùng ví_dụ strongyloides_stercoralis những loại gây bệnh_nhiễm giun_móc virus papilloma người có_thể xâm_nhập vào da bình_thường gây ra mụn cóc một_số ký_sinh_trùng ví_dụ strongyloides stercoralis những loại gây bệnh nhiễmgiun móc"
tokens = tokenizer.tokenize(test_sentence)
print(tokens)

['da</w>', 'thường</w>', 'chặn</w>', 'các</w>', 'vi_sinh_vật', 'xâm_nhập', 'trừ</w>', 'khi</w>', 'nó</w>', 'bị</w>', 'tổn_thương', 'ví_dụ', 'do</w>', 'động_vật', 'chân_đốt', 'chấn_thương', 'ống</w>', 'thông</w>', 'iv</w>', 'hoặc</w>', 'phẫu_thuật', 'rạch</w>', '.</w>', 'các</w>', 'ngoại_lệ', 'bao_gồm', 'virus</w>', 'papilloma_người', 'có_thể', 'xâm_nhập', 'vào</w>', 'da</w>', 'bình_thường', 'gây</w>', 'ra</w>', 'mụn</w>', 'cóc</w>', 'một_số', 'ký_sinh_trùng', 'ví_dụ', 'strongyloides_stercoralis', 'những</w>', 'loại</w>', 'gây</w>', 'bệnh_nhiễm', 'giun_móc', 'virus</w>', 'pap', 'illo', 'ma</w>', 'người</w>', 'có_thể', 'xâm_nhập', 'vào</w>', 'da</w>', 'bình_thường', 'gây</w>', 'ra</w>', 'mụn</w>', 'cóc</w>', 'một_số', 'ký_sinh_trùng', 'ví_dụ', 'stro', 'ng', 'y', 'loides</w>', 'ster', 'cor', 'alis</w>', 'những</w>', 'loại</w>', 'gây</w>', 'bệnh</w>', 'nhiễ', 'm', 'giun</w>', 'móc</w>']


## 5. Train-Val Split

In [60]:
# df = df.head(10000)

In [61]:
# df.shape

In [62]:
from sklearn.model_selection import train_test_split

# Chia dữ liệu thành train và validation
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

In [63]:
def preprocess_training_dataset(examples, tokenizer, max_length=256, stride=0):
    questions = [q.strip() for q in examples['Question']]
    contexts = [c.strip() for c in examples['Detailed Content']]
    answers_start = examples['Answer_Start']
    answers_text = examples['Answer']

    # Ensure tokenizer is fast and supports return_offsets_mapping
    inputs = tokenizer(
        questions,
        contexts,
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,  # Ensure this is supported
        padding="max_length",
        return_tensors="pt"
    )

    offset_mapping = inputs.pop("offset_mapping", None)
    sample_map = inputs.pop("overflow_to_sample_mapping", None)

    # Convert to list for easier handling
    if offset_mapping is not None:
        offset_mapping = [list(map(tuple, offsets)) for offsets in offset_mapping]
    if sample_map is not None:
        sample_map = list(sample_map)

    start_positions = []
    end_positions = []

    # Check if offset_mapping and sample_map are available
    if offset_mapping is not None and sample_map is not None:
        for i, offset in enumerate(offset_mapping):
            sample_idx = sample_map[i]
            if len(answers_text[sample_idx]) > 0:
                start_char = answers_start[sample_idx]
                end_char = start_char + len(answers_text[sample_idx])

                sequence_ids = inputs.sequence_ids(i)
                context_start = sequence_ids.index(1)
                context_end = len(sequence_ids) - sequence_ids[::-1].index(1) - 1

                if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
                    start_positions.append(0)
                    end_positions.append(0)
                else:
                    start_idx = next((i for i in range(context_start, context_end + 1) if offset[i][0] >= start_char), 0)
                    end_idx = next((i for i in range(context_end, context_start - 1, -1) if offset[i][1] <= end_char), 0)
                    start_positions.append(start_idx)
                    end_positions.append(end_idx)
            else:
                start_positions.append(0)
                end_positions.append(0)
    else:
        # Handle cases where offset_mapping or sample_map is not available
        start_positions = [0] * len(questions)
        end_positions = [0] * len(questions)

    # Convert start and end positions to tensors
    inputs["start_positions"] = torch.tensor(start_positions, dtype=torch.long)
    inputs["end_positions"] = torch.tensor(end_positions, dtype=torch.long)

    return inputs

In [64]:
# # Tokenize và xử lý train set
# train_encodings = preprocess_training_dataset(train_df, tokenizer)

In [65]:
def preprocess_validation_dataset(examples, tokenizer, max_length, stride):
    questions = [q.strip() for q in examples["Question"] if q and isinstance(q, str)]
    contexts = [c.strip() for c in examples["Detailed Content"] if c and isinstance(c, str)]

    inputs = tokenizer(
        questions,
        contexts,
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")

    # Initialize start_positions and end_positions lists
    start_positions = []
    end_positions = []

    for i in range(len(inputs["input_ids"])):
        sequence_ids = inputs.sequence_ids(i)
        if sequence_ids is None:
            continue  # Bỏ qua nếu không có sequence_ids

        offset = inputs["offset_mapping"][i]
        if offset is None:
            continue  # Bỏ qua nếu không có offset_mapping

        # Lấy câu trả lời từ dữ liệu ví dụ
        answer = examples['Answer'][sample_map[i]]
        start_char = examples['Answer_Start'][sample_map[i]]
        end_char = start_char + len(answer)

        # Chuyển đổi các ký tự start_char và end_char thành chỉ số token
        token_start_index = 0
        while sequence_ids[token_start_index] != 1:
            token_start_index += 1

        token_end_index = len(inputs["input_ids"][i]) - 1
        while sequence_ids[token_end_index] != 1:
            token_end_index -= 1

        # Đảm bảo rằng token_start_index và token_end_index nằm trong khoảng của câu trả lời
        if not (offset[token_start_index][0] <= start_char <= offset[token_end_index][1]):
            start_positions.append(0)  # Đặt mặc định nếu không tìm thấy
            end_positions.append(0)
        else:
            while token_start_index < len(offset) and offset[token_start_index][0] <= start_char:
                token_start_index += 1
            start_positions.append(token_start_index - 1)

            while token_end_index >= 0 and offset[token_end_index][1] >= end_char:
                token_end_index -= 1
            end_positions.append(token_end_index + 1)

        # Xử lý offset_mapping cho validation
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    # Thêm example_id để ánh xạ dễ hơn
    inputs["example_id"] = [sample_map[i] for i in range(len(inputs["input_ids"]))]

    # Thêm start_positions và end_positions vào dữ liệu
    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions

    if not inputs.get("input_ids"):
        raise ValueError("No input_ids found after tokenization.")

    return inputs


## 6. Define Compute Metrics

In [66]:
# Khởi tạo metric
metric = evaluate.load("squad")

def compute_metrics(start_logits, end_logits, validation_dataset, val_df, n_best=20, max_answer_length=30):
    # Chuyển đổi validation_dataset từ PyTorch tensor về định dạng dict nếu cần
    features = validation_dataset.to_pandas().to_dict('records')  # Giả sử bạn đã chuyển đổi validation_dataset thành DataFrame và dict

    if not isinstance(features, list) or not all(isinstance(f, dict) for f in features):
        raise ValueError("`features` phải là một danh sách các từ điển.")

    # Chuyển đổi val_df thành danh sách từ điển nếu cần
    if isinstance(val_df, pd.DataFrame):
        examples = val_df.to_dict('records')
    else:
        examples = val_df

    if not isinstance(examples, list) or not all(isinstance(ex, dict) for ex in examples):
        raise ValueError("`examples` phải là một danh sách các từ điển.")

    # Tạo ánh xạ từ example_index tới các feature
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_index = feature.get("example_id", idx)  # Dùng "example_id" nếu có, nếu không thì dùng idx
        example_to_features[example_index].append(idx)

    predicted_answers = []

    # Duyệt qua từng example
    for example_index, example in tqdm(enumerate(examples)):
        context = example.get("Detailed Content", "")
        answers = []

        # Lấy tất cả feature liên quan đến example hiện tại
        for feature_index in example_to_features[example_index]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index].get("offset_mapping", [])

            # Lấy top-n start và end indexes từ logits
            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()

            # Duyệt qua các start_index và end_index để tạo câu trả lời
            for start_index in start_indexes:
                for end_index in end_indexes:
                    if start_index >= len(offsets) or end_index >= len(offsets):
                        continue
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "answer_start": offsets[start_index][0],  # Lưu start index để khớp với định dạng trả về
                    }
                    answers.append(answer)

        # Chọn câu trả lời có điểm số tốt nhất
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["text"])  # Chọn câu trả lời tốt nhất dựa trên text, điều này có thể cần điều chỉnh
            predicted_answers.append(
                {"id": example_index, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_index, "prediction_text": ""})

    # Chuẩn bị câu trả lời thực tế dựa trên example_index
    theoretical_answers = [
        {
            "id": idx,
            "answers": [
                {"text": ans["text"], "answer_start": ans["answer_start"]}
                for ans in ex.get("answers", [])
            ]
        }
        for idx, ex in enumerate(examples)
    ]

    # Tính toán metric
    return metric.compute(predictions={"id": [ans["id"] for ans in predicted_answers], "prediction_text": [ans["prediction_text"] for ans in predicted_answers]},
                         references={"id": [ans["id"] for ans in theoretical_answers], "answers": [ans["answers"] for ans in theoretical_answers]})

## 7. Custom Collate Function

In [67]:
# !pip install datasets

In [68]:
from datasets import Dataset
from torch.cuda.amp import GradScaler, autocast


In [69]:
def custom_collate_fn(features):
    batch = {}
    for k in features[0].keys():
        # Lọc ra các giá trị None hoặc không hợp lệ trước khi tạo tensor
        filtered_features = [f[k] for f in features if f[k] is not None]

        # Nếu không có dữ liệu hợp lệ, tiếp tục vòng lặp mà không tạo tensor
        if len(filtered_features) == 0:
            print(f"Warning: All features for key '{k}' are None or invalid.")
            continue

        # Kiểm tra xem các phần tử trong danh sách có phải là tensor không
        if isinstance(filtered_features[0], torch.Tensor):
            try:
                batch[k] = torch.stack(filtered_features)
            except Exception as e:
                print(f"Error stacking tensors for key '{k}': {e}")
        else:
            try:
                # Thay thế các giá trị không hợp lệ bằng giá trị mặc định
                default_value = 0  # Hoặc giá trị phù hợp với loại dữ liệu của bạn
                filtered_features = [f if isinstance(f, (int, float)) else default_value for f in filtered_features]

                # Tạo tensor từ các giá trị đã lọc
                batch[k] = torch.tensor(filtered_features, dtype=torch.long)
            except ValueError as e:
                print(f"Error converting {k} to tensor: {e}")
                print(f"Filtered features: {filtered_features}")
            except TypeError as e:
                print(f"Type error for key '{k}': {e}")
                print(f"Filtered features: {filtered_features}")
    return batch


## 8. Training Default

In [70]:
# def train_default(train_df, val_df, args):
#     # Chuyển DataFrame thành Dataset
#     train_dataset = Dataset.from_pandas(train_df)
#     val_dataset = Dataset.from_pandas(val_df)

#     print(train_df.shape)
#     print(val_df.shape)

#     print(f"Train dataset size: {len(train_dataset)}")
#     print(f"Validation dataset size: {len(val_dataset)}")
#     # Tiền xử lý dữ liệu
#     train_dataset = train_dataset.map(
#         lambda examples: preprocess_training_dataset(examples, tokenizer, args.max_length, args.stride),
#         batched=True,
#         remove_columns=train_dataset.column_names
#     )

#     val_dataset = val_dataset.map(
#         lambda examples: preprocess_validation_dataset(examples, tokenizer, args.max_length, args.stride),
#         batched=True,
#         remove_columns=val_dataset.column_names
#     )


#     # Đặt định dạng dữ liệu cho PyTorch
#     train_dataset.set_format("torch")
#     val_dataset.set_format("torch")

#     # Tạo DataLoader
#     train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True)
#     val_dataloader = DataLoader(val_dataset, batch_size=args.batch_size)

#     # Đưa mô hình về thiết bị
#     model.to(device)

#     # Khởi tạo optimizer
#     optimizer = AdamW(model.parameters(), lr=5e-5)

#     # Xác định số lượng bước (batch) trong một epoch
#     num_train_steps = len(train_dataloader)

#     # Huấn luyện mô hình
#     for epoch in range(3):  # Sử dụng số epoch mặc định là 3
#         print(f"Epoch {epoch+1}/3")

#         # Hiển thị tiến trình huấn luyện trong từng epoch
#         model.train()
#         train_loss = 0
#         progress_bar = tqdm(train_dataloader, desc=f"Training Epoch {epoch+1}", leave=False)

#         for batch in progress_bar:
#             batch = {k: v.to(device) for k, v in batch.items() if k not in ['example_id', 'token_type_ids']}
#             outputs = model(**batch)
#             loss = outputs.loss
#             loss.backward()
#             optimizer.step()
#             optimizer.zero_grad()

#             train_loss += loss.item()
#             progress_bar.set_postfix({'Loss': train_loss / (progress_bar.n + 1)})  # Hiển thị loss trung bình

#         print(f"Training Loss after epoch {epoch+1}: {train_loss / num_train_steps}")

#         # Đánh giá mô hình sau mỗi epoch
#         model.eval()
#         val_loss = 0
#         val_steps = len(val_dataloader)
#         progress_bar_val = tqdm(val_dataloader, desc=f"Validating Epoch {epoch+1}", leave=False)

#         for batch in progress_bar_val:
#           if any(v is None for v in batch.values()):  # Kiểm tra nếu có None trong batch
#               print(f"Invalid batch: {batch}")
#               continue  # Bỏ qua batch không hợp lệ

#           batch = {k: v.to(device) for k, v in batch.items() if k not in ['example_id', 'token_type_ids']}
#           with torch.no_grad():
#               outputs = model(**batch)
#               val_loss += outputs.loss.item()

#         print(f"Validation Loss after epoch {epoch+1}: {val_loss / val_steps}")

#     print("Training finished with default settings.")

## 9. Tranning

In [71]:
def convert_to_datasets(train_df, val_df):
    """Chuyển DataFrame thành Dataset"""
    train_dataset = Dataset.from_pandas(train_df)
    val_dataset = Dataset.from_pandas(val_df)
    return train_dataset, val_dataset

def preprocess_datasets(train_dataset, val_dataset, tokenizer, args):
    """Tiền xử lý dữ liệu"""

    val_dataset = val_dataset.map(
        lambda examples: preprocess_validation_dataset(examples, tokenizer, args.max_length, args.stride),
        batched=True,
        remove_columns=val_dataset.column_names,
    )

    train_dataset = train_dataset.map(
        lambda examples: preprocess_training_dataset(examples, tokenizer, args.max_length, args.stride),
        batched=True,
        remove_columns=train_dataset.column_names,
    )

    return train_dataset, val_dataset

def save_datasets(train_dataset, val_dataset, train_path, val_path):
    """Lưu Dataset thành file"""
    train_dataset.save_to_disk(train_path)
    val_dataset.save_to_disk(val_path)

def load_datasets(train_path, val_path):
    """Tải Dataset từ file"""
    train_dataset = Dataset.load_from_disk(train_path)
    val_dataset = Dataset.load_from_disk(val_path)
    return train_dataset, val_dataset

In [72]:
import torch.nn.functional as F
def train(train_df, val_df, args):
    # Đường dẫn lưu file
    train_path = '/kaggle/working/train_dataset_processed'
    val_path = '/kaggle/working/validation_dataset_processed'
    
    # Tạo thư mục để lưu mô hình
    os.makedirs(args.output_dir, exist_ok=True)
    os.makedirs(train_path, exist_ok=True)    
    os.makedirs(val_path, exist_ok=True)
    
    #     # Chuyển DataFrame thành Dataset
#     train_dataset, val_dataset = convert_to_datasets(train_df, val_df)

#     # Tiền xử lý dữ liệu
#     train_dataset, val_dataset = preprocess_datasets(train_dataset, val_dataset, tokenizer, args)

#     # Lưu Dataset thành file
#     save_datasets(train_dataset, val_dataset, train_path, val_path)

    # Tải Dataset từ file
    train_dataset, val_dataset = load_datasets(train_path, val_path)

    # Đặt định dạng dữ liệu cho PyTorch
    train_dataset.set_format("torch")
    val_dataset.set_format("torch")

    # Tạo DataLoader cho tập huấn luyện và tập validation
    train_dataloader = DataLoader(
        train_dataset,
        shuffle=True,
        collate_fn=custom_collate_fn,
        batch_size=args.batch_size,
        num_workers=4,
        pin_memory=True
    )
    eval_dataloader = DataLoader(
        val_dataset,
        collate_fn=custom_collate_fn,
        batch_size=args.batch_size,
        num_workers=4,
        pin_memory=True
    )

    # Di chuyển mô hình đến thiết bị
    model.to(device)

    # Khởi tạo optimizer và scheduler
    optimizer = AdamW(model.parameters(), lr=args.lr)
    num_update_steps_per_epoch = len(train_dataloader)
    num_training_steps = args.epochs * num_update_steps_per_epoch

    lr_scheduler = get_scheduler(
        args.scheduler,
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=num_training_steps,
    )

    # Kiểm tra checkpoint và tải nếu có
    checkpoint_path = os.path.join(args.output_dir, 'checkpoint.pth')
    start_epoch = 0
    best_loss = float('inf')  # Đặt giá trị tốt nhất ban đầu là vô cực

    if os.path.exists(checkpoint_path):
        print(f"Loading checkpoint from {checkpoint_path}...")
        checkpoint = torch.load(checkpoint_path)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        lr_scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        start_epoch = checkpoint['epoch'] + 1
        best_loss = checkpoint['best_loss']
        print(f"Checkpoint loaded. Resuming from epoch {start_epoch}")

    # Khởi tạo GradScaler cho Mixed Precision Training
    scaler = GradScaler()
    # Gradient Accumulation Steps
    accumulation_steps = args.accumulation_steps

    # Huấn luyện mô hình
    progress_bar = tqdm(range(num_training_steps))

   # Huấn luyện mô hình
    for epoch in range(start_epoch, args.epochs):
        model.train()
        optimizer.zero_grad()

        # Sử dụng tqdm để hiển thị thanh tiến độ cho mỗi epoch
        progress_bar = tqdm(total=len(train_dataloader), desc=f"Epoch {epoch + 1}/{args.epochs}", unit="batch")

        for step, batch in enumerate(train_dataloader):
            offset_mapping = batch.pop('offset_mapping', None)

            # Di chuyển dữ liệu đến thiết bị đúng
            batch = {k: v.to(device) for k, v in batch.items() if k not in ['token_type_ids', 'example_id']}

            vocab_size = model.get_input_embeddings().num_embeddings
            out_of_vocab_mask = batch['input_ids'] >= vocab_size

            if tokenizer.unk_token_id is None:
                tokenizer.add_special_tokens({'unk_token': '[UNK]'})
                unk_token_id = tokenizer.unk_token_id
                if unk_token_id is None:
                    unk_token_id = tokenizer.pad_token_id
            else:
                unk_token_id = tokenizer.unk_token_id

            if out_of_vocab_mask.any():
                batch['input_ids'] = torch.where(out_of_vocab_mask, torch.full_like(batch['input_ids'], unk_token_id), batch['input_ids'])

            # Mixed precision training
            with autocast():
                outputs = model(**batch)
                loss = outputs.loss

            # Scale loss and backward pass
            loss = loss / accumulation_steps
            scaler.scale(loss.mean()).backward()

            # Chỉ cập nhật optimizer sau mỗi accumulation_steps
            if (step + 1) % accumulation_steps == 0 or (step + 1) == len(train_dataloader):
                scaler.step(optimizer)
                scaler.update()
                lr_scheduler.step()
                optimizer.zero_grad()

            progress_bar.update(1)

        torch.cuda.empty_cache()

        # Đánh giá mô hình sau mỗi epoch
        model.eval()
        total_loss = 0.0
        criterion = torch.nn.CrossEntropyLoss()

        for batch in tqdm(eval_dataloader):
            offset_mapping = batch.pop('offset_mapping', None)
            batch = {k: v.to(device) for k, v in batch.items() if k not in ['token_type_ids', 'example_id']}

            with torch.no_grad():
                outputs = model(**batch)

            start_logits = outputs.start_logits
            end_logits = outputs.end_logits

            start_loss = criterion(start_logits, batch['start_positions'])
            end_loss = criterion(end_logits, batch['end_positions'])
            loss = (start_loss + end_loss) / 2

            total_loss += loss.item()

        avg_loss = total_loss / len(eval_dataloader)
        print(f"Epoch {epoch}: Average Loss = {avg_loss}")

        # Lưu mô hình nếu loss giảm
        if avg_loss < best_loss:
            print(f"Saving model to {args.output_dir} with loss = {avg_loss}...")
            model.save_pretrained(args.output_dir)
            optimizer_state = optimizer.state_dict()
            lr_scheduler_state = lr_scheduler.state_dict()
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer_state,
                'scheduler_state_dict': lr_scheduler_state,
                'best_loss': best_loss,
            }, checkpoint_path)
            print("Finished saving.")
            best_loss = avg_loss

## 10. Define class Args

In [73]:
class Args:
    def __init__(self):
        self.pretrained_model = 'vinai/bartpho-word-base'  # Có thể thử mô hình nhỏ hơn nếu cần hoặc vinai/bartpho-syllable, vinai/bartpho-word-base
        self.max_length = 512  # Giảm độ dài tối đa của chuỗi
        self.stride = 256  # Tăng stride để giảm chồng lấp giữa các phần chuỗi
        self.batch_size = 16  # Tăng batch size nếu GPU có đủ bộ nhớ
        self.accumulation_steps = 2  # Giảm số bước tích lũy gradient
        self.lr = 2e-5 # Tăng tốc độ học
        self.epochs = 5  # Giữ nguyên hoặc giảm nếu đã đạt hiệu suất tốt
        self.scheduler = 'linear'  # Thử scheduler nhẹ hơn
        self.metric = 'squad'
        self.output_dir = '/kaggle/working/model/fine_tuned_bartpho1'
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Instantiate args
args = Args()

## 11. Run

In [74]:
#What is the relation between# Call the training function
# train(train_df, val_df, args)
train(train_df, val_df, args)


/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/tmp/ipykernel_30/4002417439.py:67: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_

Loading checkpoint from /kaggle/working/model/fine_tuned_bartpho1/checkpoint.pth...
Checkpoint loaded. Resuming from epoch 3


/tmp/ipykernel_30/4002417439.py:76: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


  0%|          | 0/73170 [00:00<?, ?it/s]

Epoch 4/5:   0%|          | 0/14634 [00:00<?, ?batch/s]

...

Epoch 5/5:   0%|          | 0/14634 [00:00<?, ?batch/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

  0%|          | 0/3651 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Epoch 4: Average Loss = 0.5509605779425713


## 12. Save Trained Model

In [75]:
# Lưu mô hình sau khi fine-tuning
model.save_pretrained('/kaggle/working/model/fine_tuned_bartpho1')
tokenizer.save_pretrained('/kaggle/working/model/fine_tuned_bartpho1')

('/kaggle/working/model/fine_tuned_bartpho1/tokenizer_config.json',
 '/kaggle/working/model/fine_tuned_bartpho1/special_tokens_map.json',
 '/kaggle/working/model/fine_tuned_bartpho1/tokenizer.json')

## 13. Testing the Model

In [76]:
test_questions = [
    "Liệu da có vai trò gì trong rào_cản tự_nhiên chống lại nhiễm_trùng về da không?"
]

test_contexts = [
    "da thường chặn các vi_sinh_vật xâm_nhập trừ khi nó bị tổn_thương ví_dụ do động_vật chân_đốt chấn_thương ống thông iv hoặc phẫu_thuật rạch . các ngoại_lệ bao_gồm virus papilloma_người có_thể xâm_nhập vào da bình_thường gây ra mụn cóc một_số ký_sinh_trùng ví_dụ strongyloides_stercoralis những loại gây bệnh_nhiễm giun_móc virus papilloma người có_thể xâm_nhập vào da bình_thường gây ra mụn cóc một_số ký_sinh_trùng ví_dụ strongyloides stercoralis những loại gây bệnh nhiễmgiun móc"
]


In [77]:
# Tokenize câu hỏi và bối cảnh
inputs = tokenizer(
    test_questions,
    test_contexts,
    return_tensors='pt',
    padding=True,
    truncation=True,
    max_length=512
)
inputs = {key: value.to(device) for key, value in inputs.items() if key not in ['token_type_ids']}  # Lọc bỏ các key không mong muốn

# Sinh ra dự đoán từ mô hình
model.to(device)
model.eval()
with torch.no_grad():
    outputs = model(**inputs)

# Lấy logits dự đoán vị trí bắt đầu và kết thúc
start_logits = outputs.start_logits
end_logits = outputs.end_logits

# Chuyển đổi logits thành xác suất
start_probs = torch.softmax(start_logits, dim=1)
end_probs = torch.softmax(end_logits, dim=1)

# Lấy vị trí có xác suất cao nhất
start_idx = torch.argmax(start_probs, dim=1).item()
end_idx = torch.argmax(end_probs, dim=1).item()

# Trích xuất câu trả lời từ context dựa trên vị trí dự đoán
answer_tokens = inputs['input_ids'][0][start_idx:end_idx + 1]

# Giải mã các token
answer = tokenizer.convert_ids_to_tokens(answer_tokens.tolist())  # Chuyển đổi ID thành token

# Thêm dấu </w> vào cuối mỗi token nếu chưa có
answer_with_w = [token + '</w>' if not token.endswith('</w>') else token for token in answer]

# Kết hợp các token thành chuỗi câu trả lời
answer_str = ''.join(answer_with_w)

# Thay thế </w> bằng khoảng trắng cho việc in ra
formatted_answer = answer_str.replace('</w>', ' ')

# Giải mã các dự đoán
print(f"Question: {test_questions[0]}")
print(f"Answer: {formatted_answer.strip()}")


Question: Liệu da có vai trò gì trong rào_cản tự_nhiên chống lại nhiễm_trùng về da không?
Answer: da thường chặn các vi_sinh_vật xâm_nhập trừ khi nó bị tổn_thương ví_dụ do động_vật chân_đốt chấn_thương ống thông iv hoặc phẫu_thuật rạch .


# Rouge and Blue

In [78]:
# !pip install datasets

In [79]:
!pip install rouge_score

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [80]:
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
# import torch

# Hàm tính điểm ROUGE và BLEU cho câu trả lời dự đoán
def evaluate_predictions(test_questions, test_contexts, true_answers, model, tokenizer, device):
    rouge = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)  # Đổi tên biến thành `rouge` để tránh xung đột
    smooth_fn = SmoothingFunction().method4  # Smooth cho BLEU khi có n-grams nhỏ

    rouge_scores = {'rouge1': 0, 'rouge2': 0, 'rougeL': 0}
    bleu_scores = []

    # Duyệt qua tất cả các câu hỏi, ngữ cảnh và câu trả lời
    for idx, (question, context, true_answer) in enumerate(zip(test_questions, test_contexts, true_answers)):
        # Tokenize câu hỏi và ngữ cảnh
        inputs = tokenizer(question, context, return_tensors='pt', padding=True, truncation=True, max_length=512)
        inputs = {key: value.to(device) for key, value in inputs.items() if key not in ['token_type_ids']}

        # Sinh ra dự đoán từ mô hình
        model.to(device)
        model.eval()
        with torch.no_grad():
            outputs = model(**inputs)

        # Lấy logits dự đoán vị trí bắt đầu và kết thúc
        start_logits = outputs.start_logits
        end_logits = outputs.end_logits

        # Chuyển đổi logits thành xác suất
        start_probs = torch.softmax(start_logits, dim=1)
        end_probs = torch.softmax(end_logits, dim=1)

        # Lấy vị trí có xác suất cao nhất
        start_idx = torch.argmax(start_probs, dim=1).item()
        end_idx = torch.argmax(end_probs, dim=1).item()

        # Trích xuất câu trả lời từ context dựa trên vị trí dự đoán
        answer_tokens = inputs['input_ids'][0][start_idx:end_idx+1]

        # Giải mã các token
        predicted_answer_tokens = tokenizer.convert_ids_to_tokens(answer_tokens.tolist())

        # Thêm dấu </w> vào cuối mỗi token nếu chưa có
        answer_with_w = [token + '</w>' if not token.endswith('</w>') else token for token in predicted_answer_tokens]

        # Kết hợp các token thành chuỗi câu trả lời
        predicted_answer = ''.join(answer_with_w)

        # Thay thế </w> bằng khoảng trắng cho việc tính toán
        predicted_answer_cleaned = predicted_answer.replace('</w>', ' ').strip()
#         print(predicted_answer_cleaned)

        # Tính ROUGE
        rouge_score = rouge.score(true_answer, predicted_answer_cleaned)
        rouge_scores['rouge1'] += rouge_score['rouge1'].fmeasure
        rouge_scores['rouge2'] += rouge_score['rouge2'].fmeasure
        rouge_scores['rougeL'] += rouge_score['rougeL'].fmeasure

        # Tính BLEU
        reference = [true_answer.split()]
        candidate = predicted_answer_cleaned.split()
        bleu = sentence_bleu(reference, candidate, smoothing_function=smooth_fn)
        bleu_scores.append(bleu)

    # Tính điểm trung bình của ROUGE và BLEU
    num_examples = len(test_questions)
    avg_rouge_scores = {key: value / num_examples for key, value in rouge_scores.items()}
    avg_bleu_score = sum(bleu_scores) / num_examples

    return avg_rouge_scores, avg_bleu_score

# Ví dụ với tập dữ liệu test (ví dụ: val_df['Question'], val_df['Detailed Content'], val_df['Answer'])
test_questions = val_df['Question'].tolist()
test_contexts = val_df['Detailed Content'].tolist()
true_answers = val_df['Answer'].tolist()

# Áp dụng hàm để tính ROUGE và BLEU trên nhiều câu hỏi và ngữ cảnh
avg_rouge, avg_bleu = evaluate_predictions(test_questions, test_contexts, true_answers, model, tokenizer, device)

# In ra kết quả tổng quan
print(f"Average ROUGE scores: {avg_rouge}")
print(f"Average BLEU score: {avg_bleu}")


Average ROUGE scores: {'rouge1': 0.7847855114228413, 'rouge2': 0.7441483157288947, 'rougeL': 0.767822952305958}
Average BLEU score: 0.6738141929518513


In [81]:
# Use AutoTokenizer to ensure compatibility with BARTPho
model = AutoModelForQuestionAnswering.from_pretrained('/kaggle/working/model/fine_tuned_bartpho1')
tokenizer = PreTrainedTokenizerFast.from_pretrained('/kaggle/working/model/fine_tuned_bartpho1')